# Output
This notebook generates the paragraph about the microCT-scanning from logfiles of the scans.
And an XLS sheet with the details from the scans.

In [1]:
import platform
import os
import pandas
import glob
import numpy

We select a folder and go through *each* subfolder there...

In [2]:
# Load the log files from the archive
if 'Linux' in platform.system():
    BasePath = os.path.join(os.sep, 'home', 'habi', 'research-storage-uct', 'Archiv_Tape')
else:
    BasePath = os.path.join('R:\\', 'Archiv_Tape')
# Select relevant folder
Root = os.path.join(BasePath, '*Aaldijk*')
print('We are loading all the data from %s' % Root)

We are loading all the data from /home/habi/research-storage-uct/Archiv_Tape/*Aaldijk*


In [3]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [4]:
# Look for *all* log files in the selected folder
Data['LogFile'] = glob.glob(os.path.join(Root, '**', '*.log'), recursive=True)

In [5]:
# Use only the 'proj' log files
for c, row in Data.iterrows():
    if 'rec' not in row.LogFile:
        Data.drop([c], inplace=True)
    if 'rectmp' in row.LogFile:
        Data.drop([c], inplace=True)
    if 'ctan.log' in row.LogFile:
        # Remove log file from CTAn
        Data.drop([c], inplace=True)

In [6]:
# for l in Data.LogFile:
#     print(l)

In [7]:
# For the manuscript, we use only the 'Foetus02' and 'Mouse01' scans, so let's only use these logs:

In [8]:
# Use only the 'proj' log files
for c, row in Data.iterrows():
    if ('Foetus02' not in row.LogFile) & ('Mouse01' not in row.LogFile):  # Exclude all other scans
        Data.drop([c], inplace=True)
    elif 'Registration' in row.LogFile:  # We've tried to register scans with DataViewer, which we also want to exclude
        Data.drop([c], inplace=True)        

In [9]:
Data

,LogFile
1,/home/habi/research-storage-uct/Archiv_Tape/Aa...
3,/home/habi/research-storage-uct/Archiv_Tape/Aa...
9,/home/habi/research-storage-uct/Archiv_Tape/Aa...
11,/home/habi/research-storage-uct/Archiv_Tape/Aa...
12,/home/habi/research-storage-uct/Archiv_Tape/Aa...
14,/home/habi/research-storage-uct/Archiv_Tape/Aa...
17,/home/habi/research-storage-uct/Archiv_Tape/Aa...
22,/home/habi/research-storage-uct/Archiv_Tape/Aa...
24,/home/habi/research-storage-uct/Archiv_Tape/Aa...
27,/home/habi/research-storage-uct/Archiv_Tape/Aa...


In [10]:
# Generate descritive 'sample' name
Data['Sample'] = [os.path.basename(os.path.dirname(os.path.dirname(l))).split('_')[0] for l in Data['LogFile']]
# Sort 

In [11]:
import re
def scanner(logfile, verbose=False):
    hardwareversion = []
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scanner' in line:
                if verbose:
                    print(line)
                # Sometimes it's SkyScan, sometimes Skyscan, so we have to regex it :)
                machine = re.split('Sky.can', line)[1].strip()
            if 'Hardware' in line:
                if verbose:
                    print(line)
                hardwareversion = line.split('=')[1].strip()
    if hardwareversion:
        return('SkyScan %s (Version %s)' % (machine, hardwareversion))
    else:
        return('SkyScan ' + machine)    

In [12]:
def controlsoftware(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Software Ver' in line:
                if verbose:
                    print(line)
                version = line.split('=')[1].strip()
    return(version)

In [13]:
def source(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Source Ty' in line:
                if verbose:
                    print(line)
                source = line.split('=')[1].strip()
                if 'HAMAMA' in source:
                    # We split the string at '_L' to separate HAMAMATSU_L118...
                    # Afterwards we properly capitalize HAMAMATSU and
                    # join the strings back with ' L' to get the beginning of the reference back
                    source = ' L'.join([s.capitalize() for s in source.split('_L')])
    return(source)

In [14]:
def camera(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Camera T' in line or 'Camera=' in line:
                if verbose:
                    print(line)
                cam = line.split('=')[1].strip().strip(' camera')
    return(cam)

In [15]:
def voltage(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Voltage' in line:
                if verbose:
                    print(line)
                V = float(line.split('=')[1])
    return(V)

In [16]:
def current(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Source Current' in line:
                if verbose:
                    print(line)
                A = float(line.split('=')[1])
    return(A)

In [17]:
def whichfilter(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Filter=' in line:
                if verbose:
                    print(line)
                fltr = line.split('=')[1].strip().replace('  ', ' ')
                if fltr=='No Filter':
                    fltr=False
    return(fltr)

In [18]:
def numproj(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'f Files' in line:
                if verbose:
                    print(line)
                np = int(line.split('=')[1])
    return(np)

In [19]:
def stacks(logfile, verbose=False):
    with open(logfile, 'r') as f:
        # If only one stack, then there's nothing in the log file
        numstacks = 0
        for line in f:
            if 'of conn' in line:
                if verbose:
                    print(line)
                # The 'Sub-scan scan length' is listed in the log file
                # We simply select the last one, and add 1, since Bruker also starts to count at zero
                numstacks = int(line.split('=')[1])
    return(numstacks)

In [20]:
def camerasize(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Columns' in line:
                if verbose:
                    print(line)
                columns = int(line.split('=')[1])
            if 'Rows' in line:
                if verbose:
                    print(line)
                rows = int(line.split('=')[1])
    return(columns, rows)

In [21]:
def overlapscan(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Horizontal Off' in line:
                if verbose:
                    print(line)
                wide = int(line.split('=')[1])
                if wide == 1:
                    wide=False
    return(wide)

In [22]:
def threesixtyscan(logfile, verbose=False):
    threesixty = False
    with open(logfile, 'r') as f:
        for line in f:
            if '0 Rotation' in line:
                if verbose:
                    print(line)
                threesixty = line.split('=')[1].strip()
                if threesixty == 'YES':
                    threesixty = True
                else:
                    threesixty = False
    return(threesixty)

In [23]:
def rotationstep(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Rotation Step' in line:
                if verbose:
                    print(line)
                rotstep = float(line.split('=')[1])
    return(rotstep)

In [24]:
def exposure(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Exposure' in line:
                if verbose:
                    print(line)
                exp = int(line.split('=')[1])
    return(exp)

In [25]:
def averaging(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Avera' in line:
                if verbose:
                    print(line)
                details = line.split('=')[1]
                if 'ON' in details:
                    # https://stackoverflow.com/a/4894156/323100
                    avg = int(details[details.find("(")+1:details.find(")")])
                else:
                    avg=False
    return(avg)

In [26]:
import datetime
def duration(logfile, verbose=False):
    '''Returns scantime in *seconds*'''
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scan duration' in line:
                if verbose:
                    print(line)
                duration = line.split('=')[1].strip()
    # Sometimes it's '00:24:26', sometimes '0h:52m:53s' :-/
    if 'h' in duration:
        scantime = datetime.datetime.strptime(duration, '%Hh:%Mm:%Ss')
    else:
        scantime = datetime.datetime.strptime(duration, '%H:%M:%S')
    return((scantime-datetime.datetime(1900,1,1)).total_seconds())

In [27]:
def timeformat(tdelta, fmt):
    # From https://stackoverflow.com/a/8907269/323100
    d = {"days": tdelta.days}
    d["hours"], rem = divmod(tdelta.seconds, 3600)
    d["minutes"], d["seconds"] = divmod(rem, 60)
    return fmt.format(**d)

In [28]:
def pixelsize(logfile, verbose=False):
    with open(logfile, 'r') as f:
        for line in f:
            if 'Image Pixel' in line and 'Scaled' not in line:
                if verbose:
                    print(line)
                pixelsize = float(line.split('=')[1])
    return(pixelsize)

In [29]:
def version(logfile, verbose=False):
    Program = numpy.nan
    Version = numpy.nan
    with open(logfile, 'r') as f:
        for line in f:
            if 'Reconstruction Program' in line:
                if verbose:
                    print(line)
                Program = line.split('=')[1].strip()
            if 'Program Version' in line:
                if verbose:
                    print(line)
                Version = line.split('sion:')[1].strip()
    return(Program, Version)

In [30]:
def ringremoval(logfile, verbose=False):
    ring = numpy.nan
    with open(logfile, 'r') as f:
        for line in f:
            if 'Ring' in line:
                if verbose:
                    print(line)
                ring = int(line.split('=')[1].strip())
    return(ring)

In [31]:
def beamhardening(logfile, verbose=False):
    bh = numpy.nan
    with open(logfile, 'r') as f:
        for line in f:
            if 'ardeni' in line:
                if verbose:
                    print(line)
                bh = int(line.split('=')[1].strip())
    return(bh)

In [32]:
def get_reconstruction_grayvalue(logfile):
    grayvalue = None
    """How did we map the brightness of the reconstructions?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Maximum for' in line:
                grayvalue = float(line.split('=')[1])
    return(grayvalue)

In [33]:
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Software'] = [controlsoftware(log) for log in Data['LogFile']]

In [34]:
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Voxelsize_rounded'] = [round(vs,1) for vs in Data['Voxelsize']]

In [35]:
Data['Source'] = [source(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]

In [36]:
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]

In [37]:
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['NumberOfProjections'] = [numproj(log) for log in Data['LogFile']]
Data['CameraSize'] = [camerasize(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Wide'] = [overlapscan(log) for log in Data.LogFile]
Data['ThreeSixtyScan'] = [threesixtyscan(log) for log in Data['LogFile']]

In [38]:
Data['RingRemoval'] = [ringremoval(log) for log in Data['LogFile']]
Data['Beamhardening'] = [beamhardening(log) for log in Data['LogFile']]
Data['GrayValueMax'] = [get_reconstruction_grayvalue(log) for log in Data['LogFile']]

In [39]:
Data['Exposure'] = [exposure(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]

In [40]:
Data['Duration'] = [duration(log) for log in Data['LogFile']]

In [41]:
Data['Version'] = [version(log) for log in Data['LogFile']]

In [42]:
def get_scandate(logfile, verbose=False):
    """When did we scan the Sample?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Study Date and Time' in line:
                if verbose:
                    print('Found "date" line: %s' % line.strip())
                datestring = line.split('=')[1].strip().replace('  ', ' ')
                if verbose:
                    print('The date string is: %s' % datestring)
                date = pandas.to_datetime(datestring , format='%d %b %Y %Hh:%Mm:%Ss')
                if verbose:
                    print('Parsed to: %s' % date)
    return(date)

In [43]:
Data['Scan date'] = [get_scandate(log) for log in Data['LogFile']]
# Calculate time 'spent' since start (for each sample separately)
# Data['Time passed'] = [sd - Data['Scan date'].min() for sd in Data['Scan date']]
for sample in Data.Sample.unique():
    for c, row in Data[Data.Sample == sample].iterrows():
        Data.at[c, 'Time passed'] = row['Scan date'] - Data[Data.Sample == sample]['Scan date'].min()
# Also extract days, rounded
Data['Days passed'] = [t.round('d') for t in Data['Time passed']]

In [44]:
# Sort our dataframe by Sample and time passed
Data.sort_values(['Sample', 'Time passed'], inplace=True)

In [45]:
for i in Data:
    print("'%s'," % i)

'LogFile',
'Sample',
'Scanner',
'Software',
'Voxelsize',
'Voxelsize_rounded',
'Source',
'Camera',
'Voltage',
'Current',
'Filter',
'Stacks',
'NumberOfProjections',
'CameraSize',
'RotationStep',
'Wide',
'ThreeSixtyScan',
'RingRemoval',
'Beamhardening',
'GrayValueMax',
'Exposure',
'Averaging',
'Duration',
'Version',
'Scan date',
'Time passed',
'Days passed',


In [46]:
Data[['LogFile',
      'Sample',
      'Scanner',
      'Software',
      'Voxelsize',
      'Voxelsize_rounded',
      'Source',
      'Camera',
      'Voltage',
      'Current',
      'Filter',
      'Stacks',
      'NumberOfProjections',
      'CameraSize',
      'RotationStep',
      'Wide',
      'ThreeSixtyScan',
      'RingRemoval',
      'Beamhardening',
      'GrayValueMax',
      'Exposure',
      'Averaging',
      'Duration',
      'Version',
      'Scan date',
      'Time passed',
      'Days passed']].to_csv(os.path.join('data', 'ScanningDetails.csv'))

In [47]:
Data

,LogFile,Sample,Scanner,Software,Voxelsize,Voxelsize_rounded,Source,Camera,Voltage,Current,...,RingRemoval,Beamhardening,GrayValueMax,Exposure,Averaging,Duration,Version,Scan date,Time passed,Days passed
38,/home/habi/research-storage-uct/Archiv_Tape/Aa...,Foetus02,SkyScan 2214,1.8,20.000477,20.0,Hamamatsu L10711,"FlatPanel: 2...50um, 140mm max.FOV",85.0,120.0,...,0,0,0.055,850,False,6678.0,"(NRecon, 2.1.0.1)",2021-10-25 18:46:14,0 days 00:00:00,0 days
27,/home/habi/research-storage-uct/Archiv_Tape/Aa...,Foetus02,SkyScan 2214,1.8,20.000477,20.0,Hamamatsu L10711,"FlatPanel: 2...50um, 140mm max.FOV",80.0,120.0,...,0,0,0.055,950,False,6643.0,"(NRecon, 2.1.0.1)",2021-11-08 16:12:19,13 days 21:26:05,14 days
17,/home/habi/research-storage-uct/Archiv_Tape/Aa...,Foetus02,SkyScan 2214,1.8,20.000477,20.0,Hamamatsu L10711,"FlatPanel: 2...50um, 140mm max.FOV",80.0,120.0,...,0,0,0.055,1450,False,9394.0,"(NRecon, 2.1.0.1)",2021-11-15 17:32:52,20 days 22:46:38,21 days
22,/home/habi/research-storage-uct/Archiv_Tape/Aa...,Foetus02,SkyScan 2214,1.8,20.000477,20.0,Hamamatsu L10711,"FlatPanel: 2...50um, 140mm max.FOV",80.0,120.0,...,0,0,0.055,1482,False,6319.0,"(NRecon, 2.1.0.1)",2021-11-26 12:25:21,31 days 17:39:07,32 days
82,/home/habi/research-storage-uct/Archiv_Tape/Aa...,Foetus02,SkyScan 2214,1.8,20.000477,20.0,Hamamatsu L10711,"FlatPanel: 2...50um, 140mm max.FOV",80.0,120.0,...,0,0,0.055,873,False,4651.0,"(NRecon, 2.1.0.1)",2021-12-15 17:27:57,50 days 22:41:43,51 days
53,/home/habi/research-storage-uct/Archiv_Tape/Aa...,Foetus02,SkyScan 2214,1.8,20.000477,20.0,Hamamatsu L10711,"FlatPanel: 2...50um, 140mm max.FOV",80.0,120.0,...,0,0,0.055,873,False,4675.0,"(NRecon, 2.1.0.1)",2021-12-24 11:00:51,59 days 16:14:37,60 days
1,/home/habi/research-storage-uct/Archiv_Tape/Aa...,Foetus02,SkyScan 2214,1.8,20.000477,20.0,Hamamatsu L10711,"FlatPanel: 2...50um, 140mm max.FOV",80.0,120.0,...,0,0,0.055,873,False,4641.0,"(NRecon, 2.1.0.1)",2021-12-31 10:56:23,66 days 16:10:09,67 days
51,/home/habi/research-storage-uct/Archiv_Tape/Aa...,Foetus02,SkyScan 2214,1.8,20.000477,20.0,Hamamatsu L10711,"FlatPanel: 2...50um, 140mm max.FOV",80.0,120.0,...,0,0,0.055,900,False,4736.0,"(NRecon, 2.1.0.1)",2022-01-10 11:27:14,76 days 16:41:00,77 days
40,/home/habi/research-storage-uct/Archiv_Tape/Aa...,Foetus02,SkyScan 2214,1.8,20.000477,20.0,Hamamatsu L10711,"FlatPanel: 2...50um, 140mm max.FOV",80.0,120.0,...,0,0,0.055,900,False,4756.0,"(NRecon, 2.1.0.1)",2022-01-17 11:23:53,83 days 16:37:39,84 days
3,/home/habi/research-storage-uct/Archiv_Tape/Aa...,Foetus02,SkyScan 2214,1.8,20.000477,20.0,Hamamatsu L10711,"FlatPanel: 2...50um, 140mm max.FOV",80.0,120.0,...,0,0,0.055,900,False,4740.0,"(NRecon, 2.1.0.1)",2022-01-28 10:50:39,94 days 16:04:25,95 days


----

My microct blurb from http://simp.ly/publish/NBhZhH

In [48]:
print('Based on the %s log files read from %s' % (len(Data), Root))

Based on the 33 log files read from /home/habi/research-storage-uct/Archiv_Tape/*Aaldijk*


In [49]:
print('After $PREPARATION, the',
      len(Data.Sample.unique()),
      'samples were imaged on a Bruker',
      " OR ".join(str(value) for value in Data.Scanner.unique()),
      'high-resolution microtomography machine (Control software version',
      " OR ".join(str(value) for value in Data.Software.unique()) + 
      ', Bruker microCT, Kontich, Belgium).')

After $PREPARATION, the 2 samples were imaged on a Bruker SkyScan 2214 high-resolution microtomography machine (Control software version 1.8, Bruker microCT, Kontich, Belgium).


In [50]:
print('The machine is equipped with a',
      " OR ".join(str(value) for value in Data.Source.unique()),
      'X-ray source and a',
      " OR ".join(str(value) for value in Data.Camera.unique()),
      'camera.')

The machine is equipped with a Hamamatsu L10711 X-ray source and a FlatPanel: 2...50um, 140mm max.FOV camera.


In [51]:
for sample in Data.Sample.unique():
    print('For the "%s" scans' % sample)
    print('We scanned at %s different time points' % len(Data[Data.Sample == sample]['Days passed'].unique()))
    print('Over a total of %s days' % Data[Data.Sample == sample]['Days passed'].max().days)
    print('I.e. on average one scan every %s days' % round(Data[Data.Sample == sample]['Days passed'].max().days / len(Data[Data.Sample == sample]['Days passed'].unique())))
    print(80 * '-')

For the "Foetus02" scans
We scanned at 14 different time points
Over a total of 158 days
I.e. on average one scan every 11 days
--------------------------------------------------------------------------------
For the "Mouse01" scans
We scanned at 16 different time points
Over a total of 162 days
I.e. on average one scan every 10 days
--------------------------------------------------------------------------------


In [52]:
for sample in Data.Sample.unique():
    print('For the "%s" scans' % sample)
    print('The reslulting voxel size was %s μm' % Data[Data.Sample == sample].Voxelsize.unique())
    print('The X-ray source was set to a tube voltage of', 
          " OR ".join(str(value) for value in Data[Data.Sample == sample].Voltage.unique()),
          'kV and a tube current of',
          " OR ".join(str(value) for value in Data[Data.Sample == sample].Current.unique()),
          'µA, the x-ray spectrum was', end=' ')
    if Data[Data.Sample == sample].Filter.unique():
        print('filtered by', " OR ".join(str(value) for value in Data[Data.Sample == sample].Filter.unique()), end='.')
    else:
        print('not filtered', end='.')
    print()
    print(80 * '-')

For the "Foetus02" scans
The reslulting voxel size was [20.000477 10.999632] μm
The X-ray source was set to a tube voltage of 85.0 OR 80.0 kV and a tube current of 120.0 µA, the x-ray spectrum was filtered by Al 1mm.
--------------------------------------------------------------------------------
For the "Mouse01" scans
The reslulting voxel size was [15.000176] μm
The X-ray source was set to a tube voltage of 60.0 kV and a tube current of 140.0 µA, the x-ray spectrum was not filtered.
--------------------------------------------------------------------------------


In [53]:
for sample in Data.Sample.unique():
    print('For the "%s" scans' % sample)
    for ts in Data[Data.Sample == sample].ThreeSixtyScan.unique():
        print('For 360°==%s' % ts)
        print(Data[Data.Sample == sample][Data[Data.Sample == sample].ThreeSixtyScan == ts][['Voxelsize', 'Stacks',
                                                                                             'CameraSize', 'NumberOfProjections', 'RotationStep',
                                                                                             'Exposure', 'Duration']])
        print(80*'-')

For the "Foetus02" scans
For 360°==False
    Voxelsize  Stacks    CameraSize  NumberOfProjections  RotationStep  \
38  20.000477       0  (3072, 1944)                 2205          0.10   
27  20.000477       0  (3072, 1944)                 2205          0.10   
17  20.000477       0  (3072, 1944)                 2205          0.10   
22  20.000477       0  (3072, 1944)                 1471          0.15   
82  20.000477       0  (3072, 1944)                 1471          0.15   
53  20.000477       0  (3072, 1944)                 1471          0.15   
1   20.000477       0  (3072, 1944)                 1471          0.15   
51  20.000477       0  (3072, 1944)                 1471          0.15   
40  20.000477       0  (3072, 1944)                 1471          0.15   
3   20.000477       0  (3072, 1944)                 1471          0.15   
44  20.000477       0  (3072, 1944)                 1471          0.15   
9   20.000477       0  (3072, 1944)                 1471          0.15 

In [54]:
for vs in Data.Voxelsize.unique():
    print(vs)

20.000477
10.999632
15.000176


In [55]:
for vs in Data.Voxelsize.unique():
    print('For each sample scanned with %s μm, we recorded a set of either' % round(vs), end=' ')
    if Data[Data.Voxelsize == vs].Filter.unique().tolist()[0]:   
        print(" or ".join(str(value) for value in Data[Data.Voxelsize == vs].Stacks.unique()),
              'stacked scans overlapping the sample height, each stack was recorded with', end=' ')
    print(" or ".join(str(value) for value in Data[Data.Voxelsize == vs].NumberOfProjections.unique()), 'projections with a size of', end=' ')
    for cs in Data[Data.Voxelsize == vs].CameraSize.unique():
        print(cs[0], end=' ')
    print('x', end=' ')
    for cs in Data[Data.Voxelsize == vs].CameraSize.unique():
        print(cs[1], end=' ')
    print('pixels', end=' ')
    if Data[Data.Voxelsize == vs].Wide.unique().tolist()[0]:
        print('(' + " or ".join(str(value) for value in Data[Data.Voxelsize == vs].Wide.unique()), 'projections stitched laterally)', end=' ')
    print('at every',
           str(" or ".join(str(value) for value in Data[Data.Voxelsize == vs].RotationStep.unique())) + '° over ', end='')
    if len(Data[Data.Voxelsize == vs].ThreeSixtyScan.unique()) > 1:
         print('either 180° or 360°', end=' ')
    else:
        if Data[Data.Voxelsize == vs].ThreeSixtyScan.unique()[0]:
            print('360°', end=' ')
        else:
            print('180°', end=' ')
    print('sample rotation.')
    print(80*'-')

For each sample scanned with 20 μm, we recorded a set of either 0 stacked scans overlapping the sample height, each stack was recorded with 2205 or 1471 projections with a size of 3072 x 1944 pixels at every 0.1 or 0.15° over 180° sample rotation.
--------------------------------------------------------------------------------
For each sample scanned with 11 μm, we recorded a set of either 3 stacked scans overlapping the sample height, each stack was recorded with 3601 projections with a size of 3072 x 1944 pixels at every 0.1° over 360° sample rotation.
--------------------------------------------------------------------------------
For each sample scanned with 15 μm, we recorded a set of either 2205 projections with a size of 3072 2457 x 1944 1944 pixels at every 0.1° over 180° sample rotation.
--------------------------------------------------------------------------------


In [56]:
for vs in Data.Voxelsize.unique():
    print('For each sample scanned with %s μm we exposed every single projection for (on average)' % round(vs), end=' ')
    print('%s ms' % round(Data[Data.Voxelsize == vs].Exposure.mean()))
    print(80*'-')

For each sample scanned with 20 μm we exposed every single projection for (on average) 975 ms
--------------------------------------------------------------------------------
For each sample scanned with 11 μm we exposed every single projection for (on average) 900 ms
--------------------------------------------------------------------------------
For each sample scanned with 15 μm we exposed every single projection for (on average) 979 ms
--------------------------------------------------------------------------------


In [57]:
for vs in Data.Voxelsize.unique():
    print('This resulted in a average scan time of ', end='')
    print(timeformat(datetime.timedelta(seconds=Data[Data.Voxelsize == vs].Duration.mean()),
                     '{hours} hours and {minutes} minutes for each of the *stacks*'))
    print(80*'-')

This resulted in a average scan time of 1 hours and 33 minutes for each of the *stacks*
--------------------------------------------------------------------------------
This resulted in a average scan time of 3 hours and 7 minutes for each of the *stacks*
--------------------------------------------------------------------------------
This resulted in a average scan time of 2 hours and 2 minutes for each of the *stacks*
--------------------------------------------------------------------------------


In [58]:
print('The projection images were then subsequently reconstructed into a 3D stack',
      'of images with',
      Data.Version.unique()[0][0],
      '(Version',
      Data.Version.unique()[0][1] + ', Bruker microCT, Kontich Belgium).')

The projection images were then subsequently reconstructed into a 3D stack of images with NRecon (Version 2.1.0.1, Bruker microCT, Kontich Belgium).


In [59]:
print('The whole process resulted in datasets with an isometric voxel size of',
      " or ".join(str(value) for value in Data.Voxelsize_rounded.unique()),
      'µm.') 

The whole process resulted in datasets with an isometric voxel size of 20.0 or 11.0 or 15.0 µm.
